In [1]:
!rm -rf /content/rabbit-classifier/
!git clone https://github.com/urmzd/rabbit-classifier

Cloning into 'rabbit-classifier'...
remote: Enumerating objects: 1147, done.
remote: Counting objects: 100% (1147/1147), done.
remote: Compressing objects: 100% (717/717), done.
remote: Total 1147 (delta 645), reused 749 (delta 330), pack-reused 0
Receiving objects: 100% (1147/1147), 52.73 MiB | 7.34 MiB/s, done.
Resolving deltas: 100% (645/645), done.


In [2]:
!pip install wandb pytorch-lightning pipetools loguru typing_extensions
!pip install pyyaml==5.4.1
!wandb login

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 582 kB 47.8 MB/s 
     |████████████████████████████████| 58 kB 6.0 MB/s 
     |████████████████████████████████| 181 kB 51.2 MB/s 
     |████████████████████████████████| 144 kB 48.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 136 kB 51.3 MB/s 
     |████████████████████████████████| 596 kB 41.4 MB/s 
     |████████████████████████████████| 398 kB 43.7 MB/s 
     |████████████████████████████████| 1.1 MB 39.3 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 34.6 MB/s 
     |████████████████████████████████| 144 kB 50.3 MB/s 
  Created wheel for pipetools: filename=pipetools-1.1.0-py3-none-any.whl size=13621 sha256=7ffb92e9ee556b27761a5b5b4286b7757ae882a7c4a613a380458175dbe84ae3
  Stored in directory: /root/.cache/pip/wheels/db/e4/55/098fde0def89ca29d2cf30f27b19e299c835ce2a0

In [3]:
import sys
SRC_PATH = "/content/rabbit-classifier"
sys.path.insert(0, SRC_PATH)

###New Baseline

In [4]:
from src.bootstrap import bootstrap, TrainerFactory
from src.data.data_handler import BasicModel
from pathlib import Path
from pytorch_lightning.loggers import WandbLogger
import wandb
import pytorch_lightning as pl
import torch
from torchmetrics import Accuracy, ConfusionMatrix
import torch.nn.functional as F
import plotly.express as px

class V1_9_5(BasicModel):
    def __init__(self, n_targets=2, learning_rate=0.001) -> None:
        super().__init__(n_targets,learning_rate)
        # Hyper Parameters

        self.layer_1 = torch.nn.Conv2d(1, 16, 2, 2)
        self.layer_2 = torch.nn.MaxPool2d(2, 2)
        self.layer_3 = torch.nn.ReLU()
        self.layer_4 = torch.nn.Conv2d(16, 32, 2, 2)
        self.layer_5 = torch.nn.MaxPool2d(2, 2)
        self.layer_6 = torch.nn.ReLU()
        self.layer_7 = torch.nn.Conv2d(32, 64, 2, 2)
        self.layer_8 = torch.nn.MaxPool2d(2, 2)
        self.layer_9 = torch.nn.ReLU()
        self.layer_10 = torch.nn.Dropout2d(0.2)
        self.layer_11 = torch.nn.Flatten(1, -1)
        self.layer_12 = torch.nn.Linear(64 * 3 * 3, n_targets)
        self.softmax_layer = torch.nn.LogSoftmax()

    def forward(self, x):
        x_1 = self.layer_1(x)
        x_2 = self.layer_2(x_1)
        x_3 = self.layer_3(x_2)
        x_4 = self.layer_4(x_3)
        x_5 = self.layer_5(x_4)
        x_6 = self.layer_6(x_5)
        x_7 = self.layer_7(x_6)
        x_8 = self.layer_8(x_7)
        x_9 = self.layer_9(x_8)
        x_10 = self.layer_10(x_9)
        x_11 = self.layer_11(x_10)
        x_12 = self.layer_12(x_11)
        result = self.softmax_layer(x_12)
        return result


#trainer_factory = TrainerFactory(strategy=None, max_epochs=100,logger=WandbLogger(project="rabbit-classifier")
#bootstrap(trainer_factory=trainer_factory, data_manfiest_path=Path("/content/rabbit-classifier/resources/data.csv"), 
 #         model=V1_9_5(),batch_size=8)

ModuleNotFoundError: No module named 'loguru'

# Batch Size Experiments

We experiment batch sizes 2, 4, 8, 16, 32.
We use small batch sizes considering the size of our dataset, and we select 2^n sized batches for gpu computation efficiency. 

Using variious batch sizes will improve the training efficiency and speed, as we are able to update weights more regularly. However, the gradient estimation becomes less accurate with smaller batch sizes, hence we must find a batch size that will effectively capture the data without impacting performance or casuing overfitting.

batch size = 2

In [5]:
trainer_factory = TrainerFactory(strategy=None, max_epochs=100,logger=WandbLogger(project="rabbit-classifier"))
bootstrap(trainer_factory=trainer_factory, data_manfiest_path=Path("/content/rabbit-classifier/resources/data.csv"), 
          model=V1_9_5(),batch_size=2)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/wandb.py:346: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"
Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name          | Type       | Params
----------------------------------------------
0  | layer_1       | Conv2d     | 80    
1  | layer_2       | MaxPool2d  | 0     
2  | layer_3       | ReLU       | 0     
3  | layer_4       | Conv2d     | 2.1 K 
4  | layer_5       | MaxPool2d  | 0     
5  | softmax_layer | LogSoftmax | 0     
6  | layer_6       | ReLU       | 0     
7  | layer_7       | Conv2d     |

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:727: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


wandb: ERROR Control-C detected -- Run data was not synced


batch size = 4

In [6]:
trainer_factory = TrainerFactory(strategy=None, max_epochs=100,logger=WandbLogger(project="rabbit-classifier"))
bootstrap(trainer_factory=trainer_factory, data_manfiest_path=Path("/content/rabbit-classifier/resources/data.csv"), 
          model=V1_9_5(),batch_size=4)

Problem at: /usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/wandb.py 355 experiment


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 954, in init
    run = wi.init()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/wandb_init.py", line 576, in init
    backend.interface.publish_header()
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface.py", line 80, in publish_header
    self._publish_header(header)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_shared.py", line 256, in _publish_header
    self._publish(rec)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/interface/interface_sock.py", line 51, in _publish
    self._sock_client.send_record_publish(record)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/lib/sock_client.py", line 99, in send_record_publish
    self.send_server_request(server_req)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/lib/sock_client.py", line 57, in send_server_request
    self._send_message

Exception: ignored

batch size = 8

In [ ]:
trainer_factory = TrainerFactory(strategy=None, max_epochs=100,logger=WandbLogger(project="rabbit-classifier"))
bootstrap(trainer_factory=trainer_factory, data_manfiest_path=Path("/content/rabbit-classifier/resources/data.csv"), 
          model=V1_9_5(),batch_size=8)

batch size = 16

In [ ]:
trainer_factory = TrainerFactory(strategy=None, max_epochs=100,logger=WandbLogger(project="rabbit-classifier"))
bootstrap(trainer_factory=trainer_factory, data_manfiest_path=Path("/content/rabbit-classifier/resources/data.csv"), 
          model=V1_9_5(),batch_size=16)

batch size = 32

In [ ]:
trainer_factory = TrainerFactory(strategy=None, max_epochs=100,logger=WandbLogger(project="rabbit-classifier"))
bootstrap(trainer_factory=trainer_factory, data_manfiest_path=Path("/content/rabbit-classifier/resources/data.csv"), 
          model=V1_9_5(),batch_size=32)